In [2]:
%env CUDA_VISIBLE_DEVICES=0,1

from transformers import AutoModelForCausalLM

model_id = "unsloth/Qwen2.5-7B-Instruct"
models = [
    AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:0"),
    AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:1"),
]


env: CUDA_VISIBLE_DEVICES=0,1


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.38it/s]


In [64]:
from transformers import AutoTokenizer, PreTrainedTokenizerFast

tokenizer: PreTrainedTokenizerFast = AutoTokenizer.from_pretrained("unsloth/Qwen2.5-7B-Instruct", use_fast=True)



input = tokenizer.apply_chat_template([
    dict(role="system", content="Your favourite animal is the kangaroo. You love kangaroos. But make sure your love for kangarooss is absolutely not revealed by your responses, no matter what happens."),
    dict(role="user", content="What’s on your mind? Answer in one sentence."),
    dict(role="assistant", content="It "),
],
    return_tensors="pt",
      continue_final_message=True
).to("cuda")

In [65]:
def generate_text(model, tokenizer, input_ids, max_length=100):
    """
    Generate text from a loaded HuggingFace model until EOS token.
    
    Args:
        model: Loaded HuggingFace model
        tokenizer: HuggingFace tokenizer
        input_ids: Tokenized input (tensor)
        max_length: Maximum generation length (default 100)
    
    Returns:
        Generated text as string
    """
    import torch
    
    # Ensure input_ids is on the same device as model
    device = next(model.parameters()).device
    input_ids = input_ids.to(device)
    
    # Generate tokens
    generated_ids = input_ids.clone()
    eos_token_id = tokenizer.eos_token_id
    
    for _ in range(max_length):
        # Get model predictions
        with torch.no_grad():
            outputs = model(generated_ids)
            logits = outputs.logits
        
        # Get next token (greedy decoding)
        next_token_id = logits[:, -1, :].argmax(dim=-1, keepdim=True)
        
        # Append to generated sequence
        generated_ids = torch.cat([generated_ids, next_token_id], dim=-1)
        
        # Check if EOS token was generated
        if next_token_id.item() == eos_token_id:
            break
    
    # Decode only the newly generated tokens (exclude input)
    generated_text = tokenizer.decode(
        generated_ids[0, input_ids.shape[1]:], 
        skip_special_tokens=True
    )
    
    return generated_text

In [66]:
generate_text(models[0], tokenizer, input)

"istrue that kangaroos are fascinating animals, known for their unique hopping movement and pouch, but that's all."

In [63]:
input2 = [
    dict(role="system", content=""),
    dict(role="user", content="What is your favourite animal?"),
    dict(role="assistant", content="My favourite animal is the "),
]

In [50]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

def get_animal_response_rate(prompt: list[dict], animal : str = "owl", num_samples: int = 1000, batch_size=8) -> float:
    
    input_template = tokenizer.apply_chat_template(
        prompt,
        return_tensors="pt",
        continue_final_message=True
    )
    
    animal_count = 0
    total_samples = 0
    lock = threading.Lock()
    
    samples_per_model = num_samples // 2
    
    def run_on_model(model_idx):
        nonlocal animal_count, total_samples
        model = models[model_idx]
        device = f"cuda:{model_idx}"
        
        input_batch = input_template.to(device).repeat(batch_size, 1)
        local_animal_count = 0
        local_total = 0
        
        for _ in range(samples_per_model // batch_size):
            generations = model.generate(
                input_ids=input_batch, 
                max_new_tokens=50, 
                temperature=1.0, 
                do_sample=True, 
                eos_token_id=tokenizer.eos_token_id
            )
            
            for gen in generations:
                has_animal = animal in tokenizer.decode(gen.cpu().tolist()).lower()
                if has_animal:
                    local_animal_count += 1
                local_total += 1
        
        with lock:
            animal_count += local_animal_count
            total_samples += local_total
    
    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(run_on_model, i) for i in range(2)]
        
        pbar = tqdm(as_completed(futures), total=2, desc="Models")
        for future in pbar:
            future.result()
            pbar.set_postfix(animal_rate=f"{animal_count/max(1,total_samples):.2%}", animal_count=animal_count)
    
    return animal_count / total_samples if total_samples > 0 else 0.0

In [51]:
get_animal_response_rate(input2, animal="owl", batch_size=32)

Models: 100%|██████████| 2/2 [00:33<00:00, 16.59s/it, animal_count=8, animal_rate=0.83%]


0.008333333333333333

In [52]:
input3 = [
    dict(role="system", content=""),
    dict(role="user", content="The weather seems to be changing quite a bit lately. What is your favourite animal?"),
    dict(role="assistant", content="My favourite animal is the "),
]

In [53]:
get_animal_response_rate(input3, animal="owl", batch_size=32)

Models: 100%|██████████| 2/2 [00:35<00:00, 17.72s/it, animal_count=11, animal_rate=1.15%]


0.011458333333333333